# Get and Preprocess data from Estonian Open Data Platform

**Index:**
1. [Data acquisition](#Data-acquisition)
2. [Investigation Data Structure](#Investigating-data-structure)
3. [Converting Data](#Converting-dataset)
4. [Data analysis](#Data-analysis)
5. [Get additional Data](#Get-additional-data)
6. [Export Final Data](#Export-final-values)

### Data acquisition 

Information on the COVID-19 open data can be found on the official Terviseamet website

[Koroonaviirus SARS-CoV-2 testide avaandmete kirjeldus](https://www.terviseamet.ee/et/koroonaviirus/avaandmed)

In the section Testide avaandmete andmestruktuuri kirjeldus > Avaandmete lingid is the URL to the JSON file in which all results are published.

In [1]:
url = r'https://opendata.digilugu.ee/opendata_covid19_test_results.json' # JSON document

With the [requests](https://requests.readthedocs.io/en/master/) library the file can be downloaded and read into variable ```d```.

In [2]:
import requests 

In [3]:
r = requests.get(url)
d = r.json()

The [list](https://www.w3schools.com/python/python_lists.asp) ```d``` now contains all the results from the COVID-19 testing. The amount of tests performed can be checked with ```len()```.

In [4]:
print(len(d))

177504


In the header data for the ```requests``` response is the ```Last-Modified``` date.

In [5]:
print(r.headers['Last-Modified'])

Wed, 16 Sep 2020 07:24:55 GMT


The derived results will be collected in the [dictionary](https://www.w3schools.com/python/python_dictionaries.asp) ```dc```.

In [6]:
dc = {}

In [7]:
#from datetime import datetime
from dateutil import tz
from dateutil.parser import parse
tzone = tz.gettz('Europe/Tallinn')

In [8]:
dc['totalTested'] = len(d)
dc['lastUpdate'] = parse(r.headers['Last-Modified']).astimezone(tzone).strftime('%d.%m.%Y %H:%M:%S')

### Investigating data structure

We can access the indivudal entries of the list ```d```. The first one with ```d[0]``` or the last one with ```d[-1]```. The list items are of type dictionary.

In [11]:
d[30]

{'id': '221ff8539c2ad9eed0c02935f716307def6ad3fb518e37cf45b3b40394d0e9f7',
 'Gender': 'M',
 'AgeGroup': '25-29',
 'Country': 'Eesti',
 'County': 'Viljandi maakond',
 'ResultValue': 'N',
 'StatisticsDate': '2020-03-13',
 'ResultTime': '2020-03-12T18:00:00+02:00',
 'AnalysisInsertTime': '2020-03-13T15:23:00+02:00'}

The fields in the resulting dictionary are described on the [official website](https://www.terviseamet.ee/et/koroonaviirus/avaandmed). The fields can be accessed individually:

In [13]:
d[30]['AgeGroup']

'25-29'

### Converting dataset

The [pandas](https://pandas.pydata.org/) library can be used to get the data in a more accessible way. It provides functions for further data analysis.

In [14]:
import pandas as pd

Convert the list with dictionary items to pandas dataframe. Display the first 5 rows.

In [16]:
df = pd.DataFrame(d)
df[10:15]

,id,Gender,AgeGroup,Country,County,ResultValue,StatisticsDate,ResultTime,AnalysisInsertTime
10,7148777bb25abfcf6211575c5b4c3c64e10d9f759adf9b...,M,45-49,Eesti,Võru maakond,N,2020-03-11,2020-03-10T19:29:00+02:00,2020-03-11T13:22:53+02:00
11,ded14387a0af0168c1bc7c1bfa2ccd1a91f47eb2cb7560...,N,20-24,Eesti,Pärnu maakond,N,2020-03-11,2020-03-03T00:00:00+02:00,2020-03-11T08:49:21+02:00
12,d16f4fdef48d0ba64d75073e45d7b62ab3ec0afaf95b2b...,M,55-59,Eesti,Viljandi maakond,N,2020-03-11,2020-03-11T14:03:00+02:00,2020-03-11T17:32:21+02:00
13,7bedaddb6a0ae949ca2f86b396f7c8c4e93416046659ee...,M,60-64,Eesti,Harju maakond,N,2020-03-11,2020-03-09T21:49:00+02:00,2020-03-11T10:12:54+02:00
14,7d8c578b972490df562b9cb72c2afce24b9ee47b53bbdf...,M,15-19,Eesti,Tartu maakond,N,2020-03-11,2020-03-03T00:00:00+02:00,2020-03-11T08:48:48+02:00


#### Data cleaning

In [17]:
from aglearn import remap as rm # class has 

The ```ResultTime``` and ```AnalysisInsertTime``` are not of importance right now.

For the Maakonds an identification code is used, as the Counties have a long "Tartu maakond" and short "Tartumaa" way of spelling, which might get mixed up. The dictionary is saved in the custom ```aglearn``` library. 

Slightly adapt the Agegroup (leading zeros).

Transform the text in StatisticsDate into datetime objects.

In [18]:
df = df.drop(['ResultTime', 'AnalysisInsertTime', 'id'], axis=1)
df['MKOOD'] = df['County'].map(rm.MNIMI_MKOOD)
df['AgeGroup'] = df['AgeGroup'].map(rm.VANUSER_STR)
df['StatisticsDate'] = pd.to_datetime(df['StatisticsDate'])

In [19]:
df.head()

,Gender,AgeGroup,Country,County,ResultValue,StatisticsDate,MKOOD
0,M,10-14,Eesti,Tartu maakond,N,2020-03-10,0079
1,M,05-09,Eesti,Tartu maakond,N,2020-03-10,0079
2,M,20-24,Eesti,Harju maakond,N,2020-03-10,0037
3,M,35-39,Eesti,Tartu maakond,N,2020-03-10,0079
4,N,15-19,Eesti,Viljandi maakond,N,2020-03-10,0084


#### Export data

In [20]:
df.to_csv(r'covid_digilugu_cleaned.csv')

### Data analysis

In [21]:
dtFormat = '%d.%m.%Y'
dc['firstTest'] = df.StatisticsDate.min().strftime(dtFormat)
dc['lastTest'] = df.StatisticsDate.max().strftime(dtFormat)

#### Positive vs Negative

The function ```value_counts()``` can be used to summarize the respective columns.

In [22]:
df.ResultValue.value_counts()

N    174748
P      2756
Name: ResultValue, dtype: int64

In [23]:
dc['totalPositive'] = int(df.ResultValue.value_counts()['P'])
dc['totalNegative'] = int(df.ResultValue.value_counts()['N'])

In [24]:
dc['percPositive'] = round(dc['totalPositive']/dc['totalTested'],4)

#### Values Last day

In [25]:
from datetime import datetime as dt

In [26]:
res = df[df.StatisticsDate == dt.strptime(dc['lastTest'],dtFormat)].ResultValue.value_counts()
res

N    2396
P      36
Name: ResultValue, dtype: int64

In [27]:
if 'P' in res: # in case there are no positive results :) 
    dc['prevDayConfirmed'] = int(res['P'])
else:
    dc['prevDayConfirmed'] = 0
dc['prevDayTests'] = int(res['N'] + dc['prevDayConfirmed'])

#### Timeseries Estonia

For further statistics a one-hot encoding has to be applied on the dataset. The result is joined with the dataframe.

In [28]:
df1h = pd.get_dummies(df[['ResultValue']], prefix=['Results'])
df1h = df1h.rename(columns={'Results_N' : 'negativeTests', 'Results_P' : 'confirmedCases'})
df1h = df.join(df1h)
df1h[-10:]

,Gender,AgeGroup,Country,County,ResultValue,StatisticsDate,MKOOD,negativeTests,confirmedCases
177494,N,65-69,Eesti,Tartu maakond,N,2020-08-17,0079,1,0
177495,M,20-24,Eesti,Harju maakond,N,2020-08-17,0037,1,0
177496,N,20-24,Eesti,Pärnu maakond,N,2020-08-17,0068,1,0
177497,M,40-44,Välismaa,,N,2020-08-17,NaN,1,0
177498,N,30-34,Eesti,Saare maakond,N,2020-08-17,0074,1,0
177499,N,10-14,Eesti,Harju maakond,N,2020-08-17,0037,1,0
177500,N,60-64,Eesti,Pärnu maakond,N,2020-08-17,0068,1,0
177501,M,50-54,Eesti,Harju maakond,N,2020-08-17,0037,1,0
177502,N,60-64,Eesti,Tartu maakond,N,2020-08-17,0079,1,0
177503,N,80-84,Eesti,Rapla maakond,N,2020-08-17,0071,1,0


In [29]:
newDateRange = pd.date_range(start=dt.strptime(dc['firstTest'], dtFormat), end=dt.strptime(dc['lastTest'], dtFormat), freq='1D')

In [30]:
dfts = df1h.groupby(['StatisticsDate']).sum()
dfts['testsPerDay'] = df1h.groupby(['StatisticsDate']).count().values[:,1]
dfts = dfts.reindex(newDateRange)
dfts = dfts.fillna(0)
dfts[-10:]

,negativeTests,confirmedCases,testsPerDay
2020-09-06,695.0,16.0,711.0
2020-09-07,1700.0,32.0,1732.0
2020-09-08,1808.0,22.0,1830.0
2020-09-09,1879.0,15.0,1894.0
2020-09-10,1802.0,32.0,1834.0
2020-09-11,2172.0,23.0,2195.0
2020-09-12,1573.0,21.0,1594.0
2020-09-13,1329.0,21.0,1350.0
2020-09-14,2213.0,23.0,2236.0
2020-09-15,2396.0,36.0,2432.0


Cumulative Sums

In [31]:
dfts['cumulativeNegative'] = dfts['negativeTests'].cumsum()
dfts['cumulativePositive'] = dfts['confirmedCases'].cumsum()
dfts['testsPerformed'] = dfts['testsPerDay'].cumsum()
dfts['activeCases'] = dfts['confirmedCases'].rolling(14, min_periods=1).sum()
dfts[-10:]

,negativeTests,confirmedCases,testsPerDay,cumulativeNegative,cumulativePositive,testsPerformed,activeCases
2020-09-06,695.0,16.0,711.0,157876.0,2531.0,160407.0,256.0
2020-09-07,1700.0,32.0,1732.0,159576.0,2563.0,162139.0,269.0
2020-09-08,1808.0,22.0,1830.0,161384.0,2585.0,163969.0,274.0
2020-09-09,1879.0,15.0,1894.0,163263.0,2600.0,165863.0,275.0
2020-09-10,1802.0,32.0,1834.0,165065.0,2632.0,167697.0,289.0
2020-09-11,2172.0,23.0,2195.0,167237.0,2655.0,169892.0,292.0
2020-09-12,1573.0,21.0,1594.0,168810.0,2676.0,171486.0,303.0
2020-09-13,1329.0,21.0,1350.0,170139.0,2697.0,172836.0,322.0
2020-09-14,2213.0,23.0,2236.0,172352.0,2720.0,175072.0,325.0
2020-09-15,2396.0,36.0,2432.0,174748.0,2756.0,177504.0,341.0


Percentages 

In [ ]:
dfts['positiveTestsPerc'] = (dfts['confirmedCases' ]/dfts['testsPerDay']).round(4)
dfts['positiveTestsPercCum'] = (dfts['cumulativePositive' ]/dfts['testsPerformed']).round(4)
dfts.loc[dfts.index[-1], 'lastFeature'] = 1
dfts = dfts.reset_index().rename(columns={'index':'StatisticsDate'})
dfts

#### Timeseries Maakond

In [34]:
counties = list(df1h['County'].unique())
counties.remove('')
counties

['Tartu maakond',
 'Harju maakond',
 'Viljandi maakond',
 'Valga maakond',
 'Võru maakond',
 'Pärnu maakond',
 'Jõgeva maakond',
 'Lääne maakond',
 'Saare maakond',
 'Lääne-Viru maakond',
 'Põlva maakond',
 'Ida-Viru maakond',
 'Rapla maakond',
 'Hiiu maakond',
 'Järva maakond']

In [ ]:
i = 0
for county in counties:
    dftsm0 = df1h.loc[df1h['County'] == county] # select a subset 
    dftsm = dftsm0.groupby(['StatisticsDate']).sum() # group by date and county
    dftsm['testsPerDay'] = dftsm0.groupby(['StatisticsDate']).count().values[:,1]
    dftsm = dftsm.reindex(newDateRange)
    dftsm = dftsm.fillna(0)
    dftsm['cumulativeNegative'] = dftsm['negativeTests'].cumsum()
    dftsm['cumulativePositive'] = dftsm['confirmedCases'].cumsum()
    dftsm['testsPerformed'] = dftsm['testsPerDay'].cumsum()
    dftsm['activeCases'] = dftsm['confirmedCases'].rolling(14, min_periods=1).sum()
    dftsm.loc[dftsm.index[-1], 'lastFeature'] = 1
    dftsm['County'] = county
    #dftsm['MKOOD'] = rm.MNIMI_MKOOD[county]
    if i == 0:
        dftsm_all = dftsm
        i += 1
    else:
        dftsm_all = dftsm_all.append(dftsm)
dftsm_all['positiveTestsPerc'] = (dftsm_all['confirmedCases' ]/dftsm_all['testsPerDay']).round(4)
dftsm_all['positiveTestsPercCum'] = (dftsm_all['cumulativePositive' ]/dftsm_all['testsPerformed']).round(4)
dftsm_all = dftsm_all.reset_index().rename(columns={'index':'StatisticsDate'})
dftsm_all

#### New Cases in the Last 14 days
This value can roughly be used to estimate the number of active cases. However, it neglects the hospitalized cases, which may have a significantly longer course of healing. This number shall be considered with care.

In [36]:
from datetime import timedelta as td

In [37]:
val14d = dfts.loc[dfts['StatisticsDate'] > dfts['StatisticsDate'].max() - td(days=14)]['confirmedCases'].sum()
dc['sumLast14D'] = val14d
print('New Cases in last 14d: {}'.format(val14d))

New Cases in last 14d: 341.0


### Export Final Data

In the dictionary ```dc``` all the derived statistics are stored.

In [40]:
dc

{'totalTested': 177504,
 'lastUpdate': '16.09.2020 10:24:55',
 'firstTest': '05.02.2020',
 'lastTest': '15.09.2020',
 'totalPositive': 2756,
 'totalNegative': 174748,
 'percPositive': 0.0155,
 'prevDayConfirmed': 36,
 'prevDayTests': 2432,
 'sumLast14D': 341.0}

In [41]:
import json

In [43]:
with open(r'data/cov_stats_eesti.json', 'w') as f:
    json.dump(dc, f, indent=4)

The dataframe ```dfts``` contains the timeseries for whole Estonia.

In [ ]:
dfts

In [45]:
#dfts = dfts.reset_index()
#dfts = dfts.rename(column={})
dfts.to_csv(r'data/cov_ts_eesti.csv', index=False)

The dataframe ```dftsm_all``` contains the timeseries for each County.

In [ ]:
dftsm_all

In [47]:
dftsm_all.to_csv(r'data/ts_maakond.csv', index=False)